In [40]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import torch

import numpy as np

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('model/fine_tuned_BERT')
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [70]:
# helper functions

def preProcessInput(titles, summaries, genres):
# titles: list of strings in the form: [title_1, title_2, ...]
# summaries: list of summaries(strings) in the form: [summary_1, summary_2, ...]
# genres: list of genres in the form: [[genres_1], [genres_2], ...] with genres_i = "genres_i1", "genres_i2", ...
    
    inputs = []
    
    for i in range(len(titles)):
        # normalice spacing in the titles
        title_i = (' ').join(titles[i].split())
        
        # normalice spacing in the summaries
        summary_i = (' ').join(summaries[i].split())
        
        if genres[i] == []:
            genres_i = 'NonGiven'
        else:
            # convert the lists of genres to strings separated by '|'
            genres_i = '|'.join(genres[i])
            
        input_i = {'title': title_i, 'summary': summary_i, 'genres': genres_i}
        inputs.append(input_i)
        
    return inputs


def tokenizeInputs(inputs):
    
    title_mod = [movie['title'] + '<SEP>' + movie['summary'] for movie in inputs]
    genres_list = [movie['genres'] for movie in inputs]
    
        
    return tokenizer(title_mod, genres_list, padding = 'max_length', truncation = True, 
                     return_tensors = "pt")


def modelPredictions(model, tokenized_input):
# generate model predictions using the model logits and tokenized input and determine 
# the most likely rating using argmax
    
    with torch.no_grad():
        model_output = model(**tokenized_input)
        
    logits = model_output.logits
    predictions = np.argmax(logits, axis = -1)
    
    return predictions

# target_names = ['bad_rating', 'average_rating', 'good_rating']            
            
    

In [27]:
movies = preProcessInput(["this is the    first title", "here is another title"], 
                ["the first movie   is about icecream", "the second movie is about forests"], 
                [["Action", "Fiction"], ["Romance", "Thriller"]])

In [28]:
movies

[{'title': 'this is the first title',
  'summary': 'the first movie is about icecream',
  'genres': 'Action|Fiction'},
 {'title': 'here is another title',
  'summary': 'the second movie is about forests',
  'genres': 'Romance|Thriller'}]

In [29]:
[m['genres'] for m in movies]

['Action|Fiction', 'Romance|Thriller']

In [30]:
tokenized_input = tokenizeInputs(movies)
tokenized_input

{'input_ids': tensor([[ 101, 1142, 1110,  ...,    0,    0,    0],
        [ 101, 1303, 1110,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [31]:
type(tokenized_input.input_ids[0])

torch.Tensor

In [38]:
model(**tokenized_input)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.9529,  5.2467, -2.4924],
        [-2.3565,  3.7838, -1.5596]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [36]:
with torch.no_grad():
    
    a = model(**tokenized_input)
    a

In [44]:
logits = a.logits
logits

tensor([[-2.9529,  5.2467, -2.4924],
        [-2.3565,  3.7838, -1.5596]])

In [61]:
predictions = np.argmax(logits, axis = -1)
predictions

tensor([1, 1])

In [62]:
len(predictions)

2

In [67]:
for i in range(len(predictions)):
    print(predictions[i] == 1)

tensor(True)
tensor(True)


In [69]:
for pred in predictions:
    print(pred == 1)

tensor(True)
tensor(True)


In [71]:
modelPredictions(model, tokenized_input)

tensor([1, 1])